<a href="https://colab.research.google.com/github/JimmyCh2001/kkcompany_datagame_schubert/blob/main/kkcompany_datagame_schubert_019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello World")

Hello World


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
%cd '/content/drive/MyDrive/'

/content/drive/MyDrive


In [ ]:
!git clone https://github.com/JimmyCh2001/kkcompany_datagame_schubert.git

Cloning into 'kkcompany_datagame_schubert'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.


In [ ]:
%cd /content/drive

[Errno 2] No such file or directory: '/content/drive'
/root


In [16]:
%cd ../..

/


In [18]:
! mkdir ~/.kaggle
! cp /content/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c store-sales-time-series-forecasting

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 61% 13.0M/21.4M [00:00<00:00, 62.8MB/s]
100% 21.4M/21.4M [00:00<00:00, 85.5MB/s]


In [20]:
!mkdir /content/kkcompany_datagame_schubert/data/
!unzip store-sales-time-series-forecasting

mkdir: cannot create directory ‘/content/kkcompany_datagame_schubert/data/’: File exists
Archive:  store-sales-time-series-forecasting.zip
  inflating: holidays_events.csv     
  inflating: oil.csv                 
  inflating: sample_submission.csv   
  inflating: stores.csv              
  inflating: test.csv                
  inflating: train.csv               
  inflating: transactions.csv        


In [23]:
%cd /content/kkcompany_datagame_schubert/

/content/kkcompany_datagame_schubert


In [25]:
!git config --global user.email "chenquanhoangthang12@gmail.com"
!git config --global user.name "JimmyCh2001"

In [26]:
!git add .
!git commit -m "Test Load Data"
!git push origin main

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address
